<a href="https://colab.research.google.com/github/pradhansankalp10/SPARK-Programing/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [ ]:
spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("WordCount") \
    .getOrCreate()

file_path = "/content/spark word.txt"

text_file = spark.sparkContext.textFile(file_path)

word_counts = (
    text_file.flatMap(lambda line: line.split())
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
)

for word, count in word_counts.collect():
    print(f"{word}: {count}")

spark.stop()

Metadata: 1
is: 3
defined: 1
as: 3
“data: 1
data”—it: 1
provides: 1
describes,: 1
other: 1
data.: 2
bioinformatics,: 4
metadata: 3
large: 1
researchers: 1
understand: 1
origin,: 1
meaning: 1
of: 5
are: 2
working: 1
Metadata:: 5
•: 4
this: 2
include: 1
type: 1
sequencing: 1
platform,: 2
gene: 2
names,: 1
conditions.: 1
expression: 1
have: 1
descriptive: 1
sequenced: 2
2022-06-15”.: 1
Describes: 2
structural: 1
describe: 1
format: 1
(e.g.:: 1
FASTA,: 1
related.: 1
“FASTA: 1
format,: 1
sequence: 2
starts: 1
‘>’: 1
identifier.”: 1
when: 1
was: 3
collected,: 1
rights.: 1
“Collected: 1
XYZ: 1
Lab,: 1
xyz@lab.com,: 1
publicly: 1
CC: 1
license.”: 1
generated: 1
processed.: 1
samples,: 1
steps.: 1
“RNA: 1
extracted: 1
using: 1
poly-A: 1
an: 1
about: 3
information: 4
that: 1
explains,: 1
or: 6
gives: 1
context: 1
to: 1
In: 4
crucial: 1
for: 2
interpreting: 1
and: 4
managing: 2
biological: 3
datasets,: 1
it: 3
helps: 1
the: 12
structure,: 1
data: 3
they: 1
with.: 1
Key: 1
Aspects: 1
Descriptive: 

In [ ]:
import zipfile
import os

zip_path = '/content/football video.zip'
extraction_path = '/mnt/data/football_video/'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

os.listdir(extraction_path)


['Annotated', 'Live match images', 'madrid vs city.mp4']

In [ ]:
import zipfile
import os

zip_path = '/content/football video.zip'
extraction_path = '/content/Sankalp'

os.makedirs(extraction_path, exist_ok=True)

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extraction_path)
    print(f"Successfully unzipped to {extraction_path}")
except FileNotFoundError:
    print(f"Error: File not found at {zip_path}")
except zipfile.BadZipFile:
    print(f"Error: Invalid zip file at {zip_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

print("Files in the Sankalp directory:")
for filename in os.listdir(extraction_path):
    filename

Successfully unzipped to /content/Sankalp
Files in the Sankalp directory:


In [ ]:
import zipfile
import os
zip_file_path = '/content/football video.zip'
extraction_path = '/mnt/data/football_video/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

print("Files in extraction directory:", os.listdir(extraction_path))


Files in extraction directory: ['Annotated', 'Live match images', 'madrid vs city.mp4']


In [ ]:
video_filename = "/content/Sankalp/madrid vs city.mp4"
video_path = os.path.join(extraction_path, video_filename)


In [ ]:
import cv2

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Video file could not be opened. Check the file path and format.")
else:
    ret, frame = cap.read()
    if ret:
        print("Successfully read a frame from the video.")
    else:
        print("Error: Could not read a frame. The video might be corrupted.")
cap.release()


Successfully read a frame from the video.


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("FootballMatchAnalysis") \
    .getOrCreate()


In [ ]:
def extract_frames(video_path, frame_skip=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    frame_count = 0

    if not cap.isOpened():
        print("Error: Could not open video file.")
        return frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            frame_resized = cv2.resize(frame, (640, 360))
            _, buffer = cv2.imencode('.jpg', frame_resized)
            frames.append(buffer.tobytes())

        frame_count += 1

    cap.release()
    print(f"Extracted {len(frames)} frames from the video.")
    return frames


In [ ]:
frames = extract_frames(video_path)
if not frames:
    print("No frames extracted. Check the video path or format.")
else:
    print(f"Number of frames extracted: {len(frames)}")


Extracted 306 frames from the video.
Number of frames extracted: 306


In [ ]:
frames_rdd = spark.sparkContext.parallelize(frames)


In [ ]:
import numpy as np

def process_frame(frame_bytes):
    nparr = np.frombuffer(frame_bytes, np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_frame, 100, 200)

    edge_count = np.count_nonzero(edges)

    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    player_count = len(contours)

    return {"edge_count": edge_count, "player_count": player_count}


In [ ]:
frame_analysis = frames_rdd.map(process_frame).collect()
for idx, result in enumerate(frame_analysis):
    print(f"Frame {idx}: Edge Count = {result['edge_count']}, Player/Object Count = {result['player_count']}")


Frame 0: Edge Count = 3018, Player/Object Count = 53
Frame 1: Edge Count = 3563, Player/Object Count = 75
Frame 2: Edge Count = 3830, Player/Object Count = 65
Frame 3: Edge Count = 3689, Player/Object Count = 83
Frame 4: Edge Count = 3837, Player/Object Count = 66
Frame 5: Edge Count = 4206, Player/Object Count = 74
Frame 6: Edge Count = 15725, Player/Object Count = 337
Frame 7: Edge Count = 16842, Player/Object Count = 349
Frame 8: Edge Count = 17162, Player/Object Count = 339
Frame 9: Edge Count = 16948, Player/Object Count = 369
Frame 10: Edge Count = 17799, Player/Object Count = 411
Frame 11: Edge Count = 18671, Player/Object Count = 474
Frame 12: Edge Count = 20127, Player/Object Count = 508
Frame 13: Edge Count = 21305, Player/Object Count = 532
Frame 14: Edge Count = 21692, Player/Object Count = 495
Frame 15: Edge Count = 22986, Player/Object Count = 610
Frame 16: Edge Count = 23106, Player/Object Count = 558
Frame 17: Edge Count = 23069, Player/Object Count = 574
Frame 18: Edge

In [ ]:
spark.stop()


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

video_file_path = "/content/Sankalp/madrid vs city.mp4"

video_capture = cv2.VideoCapture(video_file_path)

if not video_capture.isOpened():
    print("Error opening video stream or file")
    exit()

while True:
    ret, frame = video_capture.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2_imshow(gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import os

output_directory = 'MATCH'
os.makedirs(output_directory, exist_ok=True)
video_file_path = "/content/Sankalp/madrid vs city.mp4"

video_capture = cv2.VideoCapture(video_file_path)

if not video_capture.isOpened():
    print("Error opening video file")
    exit()

frame_count = 0
while True:
    ret, frame = video_capture.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    output_filename = os.path.join(output_directory, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(output_filename, frame)

    frame_count += 1

video_capture.release()
print(f"Saved {frame_count} frames to the 'MATCH' directory.")

Can't receive frame (stream end?). Exiting ...
Saved 3051 frames to the 'MATCH' directory.


In [ ]:
import zipfile
import os

def zip_folder(folder_path, zip_file_name):
    try:
        with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(folder_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=os.path.relpath(file_path, folder_path))
        print(f"Successfully zipped '{folder_path}' to '{zip_file_name}'")
    except FileNotFoundError:
        print(f"Error: Folder not found at '{folder_path}'")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

folder_to_zip = 'MATCH'
zip_file_name = 'MATCH.zip'

zip_folder(folder_to_zip, zip_file_name)

Successfully zipped 'MATCH' to 'MATCH.zip'
